In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import sys
sys.path.insert(1, 'C:/Users/pnknv/Documents/GitHub/yadisk_loader')
from main_utils import *

# dadata fix

In [27]:
import random

In [28]:
a = pd.DataFrame(columns=['house_fias_id', 'data', 'geo_lat', 'geo_lon'])
b = pd.DataFrame(columns=['house_fias_id', 'data', 'geo_lat', 'geo_lon'])

In [29]:
for i in range(3):
    a.loc[len(a)] = [i, random.randint(0,1000), random.randint(0,1000), random.randint(0,1000)]

In [30]:
a

,house_fias_id,data,geo_lat,geo_lon
0,0,681,843,618
1,1,600,439,657
2,2,400,661,455


In [31]:
for i in range(6):
    if i in a.house_fias_id.unique().tolist():
        b.loc[len(b)] = a.loc[a.house_fias_id == i].values.tolist()[0]
    else:    
        b.loc[len(b)] = [i, random.randint(0,1000), random.randint(0,1000), random.randint(0,1000)]

In [32]:
b

,house_fias_id,data,geo_lat,geo_lon
0,0,681,843,618
1,1,600,439,657
2,2,400,661,455
3,3,382,734,334
4,4,477,211,102
5,5,516,747,559


In [34]:
pd.concat([a,b], ignore_index=True)

,house_fias_id,data,geo_lat,geo_lon
0,0,681,843,618
1,1,600,439,657
2,2,400,661,455
3,0,681,843,618
4,1,600,439,657
5,2,400,661,455
6,3,382,734,334
7,4,477,211,102
8,5,516,747,559


In [3]:
import json

In [4]:
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config()

In [5]:
sql_server, sql_engine = get_sql_engine()

In [6]:
def get_dadata(engine):
    dadata_query = f'SELECT id, data FROM dadata_houses'
    try:
        con_obj = engine.connect()
        dadata_db = pd.read_sql(text(dadata_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        dadata_db = None
        exc_code = exc.code
    return dadata_db, exc_code

In [ ]:
dadata = get_dadata(sql_engine)

In [ ]:
dadata = dadata[0]

In [ ]:
def get_json(json_str):
    try:
        return json.loads(json_str)
    except:
        try:
            return json.loads(json_str.replace("'", '"').replace('None', 'null'))
        except:
            try:
                return json.loads(json_str.replace('\\xa0', '').replace("'", '"').replace('None', 'null'))
            except:
                return None

In [ ]:
dadata['json_data'] = dadata.data.apply(lambda x: get_json(x))

In [ ]:
dadata[dadata.json_data.isna()]

In [ ]:
dadata['addr'] = dadata.json_data.apply(lambda x: x['source'])

In [ ]:
dadata

In [67]:
dadata.groupby(['addr'])['id'].count()

addr
Казань; 10-я Союзная ул.; 35             1
Казань; 2-я Газовая ул.; 5к2             1
Казань; 2-я Даурская ул.; 11             1
Казань; 2-я Даурская ул.; 2              2
Казань; 2-я Даурская ул.; 4к3            1
                                        ..
г. Челябинск, ш. Металлургов, д. 80А     1
г. Челябинск, ш. Металлургов, д. 82      1
г. Челябинск, ш. Металлургов, д. 82 А    1
г. Челябинск, ш. Металлургов, д. 84      1
г. Челябинск, ш. Металлургов, д. 84А     1
Name: id, Length: 138311, dtype: int64

In [73]:
addr_group = pd.DataFrame(dadata.groupby(['addr'])['id'].count().sort_values().reset_index())

In [78]:
addr_group.sort_values('id', ascending=False).to_excel('addr_group_sorted.xlsx')

In [79]:
addr_group.sort_values('id', ascending=False)

,addr,id
138310,Новосибирск; р-н Калининский; ул. Объединения;...,1159
138309,Новосибирск; пр-т Красный; стр. 236,1040
138308,Новороссийск; ул. Куникова; лит. 2,684
138307,Новосибирск; ул. Овражная; д. 2А,658
138306,Новосибирск; р-н Кировский; ул. Бородина; д. 2,509
...,...,...
46098,"г. Челябинск, ул. Южная, д. 2А",1
46097,"г. Челябинск, ул. Южная, д. 2Б",1
46096,"г. Челябинск, ул. Южная, д. 2в",1
46095,"г. Челябинск, ул. Южная, д. 3",1


In [81]:
addr_group.iloc[138310,0]

'Новосибирск; р-н Калининский; ул. Объединения; д. 102/4'